In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

In [2]:
import os

os.chdir('..') # Change to parent directory
cwd = os.getcwd() # Get current working directory
print(cwd)

C:\Users\templ\OneDrive\Desktop\AI\Hackathon\Amazon


In [3]:
# Read the dataset using Dask
dataset = pd.read_csv('dataset/trainfinal.csv', on_bad_lines='skip', encoding='utf-8', engine='python')
# dataset =  dataset.sample(n=20000)

In [4]:
dataset.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       'PRODUCT_TYPE_ID', 'PRODUCT_LENGTH'],
      dtype='object')

In [5]:
dataset

,0,1,2,3,4,5,6,7,8,9,10,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,0.033740,0.053593,0.119451,-0.002144,-0.023888,-0.009354,-0.013213,-0.011067,-0.052480,-0.005460,-0.048928,1650,2125.980000
1,0.021798,0.017311,-0.010543,0.044631,-0.013809,0.007838,-0.019112,0.012684,-0.002693,0.000218,0.007681,2755,393.700000
2,0.021713,0.010882,0.012488,-0.015654,0.011082,0.004986,-0.017478,-0.003682,-0.005930,-0.000192,-0.005423,7537,748.031495
3,0.041322,0.102819,-0.029833,0.067824,0.101342,-0.023887,0.054705,-0.005680,-0.031651,-0.000720,-0.008348,2996,787.401574
4,0.001786,0.004194,0.001742,-0.000216,-0.001976,0.006235,-0.000607,0.000235,-0.002081,0.000451,0.008637,6112,598.424000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249693,0.019544,0.061277,-0.024850,0.036257,0.073269,-0.017734,0.047925,0.000918,0.012451,-0.000118,-0.004641,3009,1181.100000
2249694,0.020147,0.015623,0.021427,-0.000404,0.003816,0.009220,-0.025823,-0.002782,-0.011322,0.000861,0.017507,3413,125.984252
2249695,0.063208,0.032880,0.119688,0.024788,-0.071651,-0.024792,0.045420,-0.009083,0.004295,-0.002458,-0.022181,1574,1200.000000
2249696,0.033887,0.054742,0.086165,-0.033040,-0.014690,-0.016088,0.023324,0.032273,0.099026,-0.000348,-0.022752,592,2900.000000


In [6]:
data = dataset

In [7]:
# Apply TF-IDF vectorization to the text data
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data.iloc[:, :10].apply(lambda x: ' '.join(x.astype(str)), axis=1))

In [8]:
X

<2249698x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 820442 stored elements in Compressed Sparse Row format>

In [10]:
# Perform dimensionality reduction with Truncated SVD
svd = TruncatedSVD(n_components=50, random_state=42)
X_svd = svd.fit_transform(X)

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_svd)

MemoryError: Unable to allocate 129. MiB for an array with shape (2249698, 60) and data type bool

In [ ]:
# # Determine the optimal number of clusters using the elbow curve method
# wcss = []
# sil_scores = []
# for i in range(2, 11):
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(X_scaled)
#     wcss.append(kmeans.inertia_)
#     sil_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# plt.plot(range(2, 11), wcss)
# plt.title('Elbow Curve')
# plt.xlabel('Number of Clusters')
# plt.ylabel('WCSS')
# plt.show()

In [ ]:
# Choose the optimal number of clusters based on the elbow curve
n_clusters = 9

# Train the K-means model with the optimal number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_scaled)

# Assign the cluster labels to the original dataset
data['cluster_label'] = kmeans.labels_

# Print the distribution of the cluster labels
print(data['cluster_label'].value_counts())